# Decision Tree Regression on Diabetes Dataset
This notebook demonstrates how to use scikit-learn's DecisionTreeRegressor for predicting disease progression. We cover:
- Dataset loading and exploration
- Key formulas (MSE, variance reduction)
- Exploratory feature-target visualization
- Training DecisionTreeRegressor (default and tuned)
- Visualizing the regression tree and prediction surface
- Model evaluation with MSE, MAE, and R²
- Hyperparameter tuning (max_depth)
- Final discussion of results

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
sns.set_theme()


## Data Loading & Exploration
Load the diabetes dataset and inspect its shape and basic statistics.

In [5]:
data = load_diabetes()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name='progression')
print(f"Feature matrix shape: {X.shape}")
X.head()

Feature matrix shape: (442, 10)


,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


In [4]:
y.describe()

count    442.000000
mean     152.133484
std       77.093005
min       25.000000
25%       87.000000
50%      140.500000
75%      211.500000
max      346.000000
Name: progression, dtype: float64